In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import KNNImputer
from sklearn.model_selection import GridSearchCV

Получение данных

In [2]:
!wget -O ../data/data.csv "https://www.dropbox.com/s/3mbvhnqm3pg8kf9/heart_cleveland_upload.csv?dl=0"

--2022-10-22 23:02:47--  https://www.dropbox.com/s/3mbvhnqm3pg8kf9/heart_cleveland_upload.csv?dl=0
Распознаётся www.dropbox.com (www.dropbox.com)… 162.125.71.18, 2620:100:6026:18::a27d:4612
Подключение к www.dropbox.com (www.dropbox.com)|162.125.71.18|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: /s/raw/3mbvhnqm3pg8kf9/heart_cleveland_upload.csv [переход]
--2022-10-22 23:02:48--  https://www.dropbox.com/s/raw/3mbvhnqm3pg8kf9/heart_cleveland_upload.csv
Повторное использование соединения с www.dropbox.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://uc1e954f449bb76910f5e421874a.dl.dropboxusercontent.com/cd/0/inline/BvUbIWdlR83Jfkh_VTSC5vqYzHYNASBLg42Xl4NpZSHDSJfABftw9vl2FeGAModYeKI7ltVHe-wunDi3NAz5x3ZYSCMj6hFnmi3uNXz5CbfO6hdAL91qiKNDmJJI-sMWWMjnUzfRMgse5dgxeck9_LAlhQqPWSMdYoR6_5-MRnajJA/file# [переход]
--2022-10-22 23:02:48--  https://uc1e954f449bb76910f5e421874a.dl.dropboxusercontent.com/cd/0/inline/BvUbIWdlR83Jfkh_VTSC

Загрузка в DataFrame

In [3]:
df = pd.read_csv("../data/data.csv")
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,condition
0,69,1,0,160,234,1,2,131,0,0.1,1,1,0,0
1,69,0,0,140,239,0,0,151,0,1.8,0,2,0,0
2,66,0,0,150,226,0,0,114,0,2.6,2,0,0,0
3,65,1,0,138,282,1,2,174,0,1.4,1,1,0,1
4,64,1,0,110,211,0,2,144,1,1.8,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,40,1,3,152,223,0,0,181,0,0.0,0,0,2,1
293,39,1,3,118,219,0,0,140,0,1.2,1,0,2,1
294,35,1,3,120,198,0,0,130,1,1.6,1,0,2,1
295,35,0,3,138,183,0,0,182,0,1.4,0,0,0,0


In [4]:
X = df.drop("condition", axis=1)
X

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,69,1,0,160,234,1,2,131,0,0.1,1,1,0
1,69,0,0,140,239,0,0,151,0,1.8,0,2,0
2,66,0,0,150,226,0,0,114,0,2.6,2,0,0
3,65,1,0,138,282,1,2,174,0,1.4,1,1,0
4,64,1,0,110,211,0,2,144,1,1.8,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,40,1,3,152,223,0,0,181,0,0.0,0,0,2
293,39,1,3,118,219,0,0,140,0,1.2,1,0,2
294,35,1,3,120,198,0,0,130,1,1.6,1,0,2
295,35,0,3,138,183,0,0,182,0,1.4,0,0,0


In [5]:
Y = df["condition"]
Y

0      0
1      0
2      0
3      1
4      0
      ..
292    1
293    1
294    1
295    0
296    1
Name: condition, Length: 297, dtype: int64

Обработка признаков

There are 13 attributes

    age: age in years
    sex: sex (1 = male; 0 = female)
    cp: chest pain type
    -- Value 0: typical angina
    -- Value 1: atypical angina
    -- Value 2: non-anginal pain
    -- Value 3: asymptomatic
    trestbps: resting blood pressure (in mm Hg on admission to the hospital)
    chol: serum cholestoral in mg/dl
    fbs: (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
    restecg: resting electrocardiographic results
    -- Value 0: normal
    -- Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
    -- Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria
    thalach: maximum heart rate achieved
    exang: exercise induced angina (1 = yes; 0 = no)
    oldpeak = ST depression induced by exercise relative to rest
    slope: the slope of the peak exercise ST segment
    -- Value 0: upsloping
    -- Value 1: flat
    -- Value 2: downsloping
    ca: number of major vessels (0-3) colored by flourosopy
    thal: 0 = normal; 1 = fixed defect; 2 = reversable defect
    and the label
    condition: 0 = no disease, 1 = disease

Категориальные из них:
    
    sex
    cp
    restecg
    exang
    slope
    thal
    
Количественные:

    age
    trestbps
    chol
    fbs
    thalach
    oldpeak
    ca
    
Целевая переменная:

    condition
    

In [6]:
cathegory_features = ["sex",
                    "cp",
                    "restecg",
                    "exang",
                    "slope",
                    "thal",]
number_features = ["age",
                "trestbps",
                "chol",
                "fbs",
                "thalach",
                "oldpeak",
                "ca",]

target_feature = "condition"

Для численных признаков - minmax скейлинг, для категориальных - one-hot

In [7]:
ct = ColumnTransformer(
     [ ("imputer", KNNImputer(), cathegory_features+number_features),
      ("number", StandardScaler(), number_features),
      ("cathegory", OneHotEncoder(handle_unknown="ignore"), cathegory_features),
    ])

In [8]:
new_df = ct.fit_transform(X)

In [9]:
pipeline = Pipeline(
    steps=[("prepare", ct),
           ("model", KNeighborsClassifier())]
)

In [10]:
parameters = {"model__n_neighbors": [2, 3, 5, 7],
              "prepare__imputer__n_neighbors": [3, 5, 7],
              "prepare__number__with_mean": [True, False],
              "prepare__number__with_std": [True, False],
              }

In [11]:
gridsearch = GridSearchCV(pipeline, parameters).fit(X, Y)

In [12]:
model = gridsearch.best_estimator_

In [13]:
gridsearch.best_params_

{'model__n_neighbors': 7,
 'prepare__imputer__n_neighbors': 3,
 'prepare__number__with_mean': True,
 'prepare__number__with_std': True}

Запуск и проверка модели

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=14565)

In [15]:
model.fit(X_train, Y_train)

Pipeline(steps=[('prepare',
                 ColumnTransformer(transformers=[('imputer',
                                                  KNNImputer(n_neighbors=3),
                                                  ['sex', 'cp', 'restecg',
                                                   'exang', 'slope', 'thal',
                                                   'age', 'trestbps', 'chol',
                                                   'fbs', 'thalach', 'oldpeak',
                                                   'ca']),
                                                 ('number', StandardScaler(),
                                                  ['age', 'trestbps', 'chol',
                                                   'fbs', 'thalach', 'oldpeak',
                                                   'ca']),
                                                 ('cathegory',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['sex', 'cp', 'restecg',
                                                   'exang', 'slope',
                                                   'thal'])])),
                ('model', KNeighborsClassifier(n_neighbors=7))])

In [16]:
Y_predicted = model.predict(X_test)
print(accuracy_score(Y_predicted, Y_test))

0.72
